<a href="https://colab.research.google.com/github/hsuanchengsun/Python-Skills/blob/master/Season_Ticket_Seat_Distribution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import useful modules

import pandas as pd
import random

In [ ]:
# Create 15000 data for season ticket holders

A = list(range(0, 15000))
B = list(random.randint(20, 129) for _ in range(15000)) # I use the range(20, 129) as the mock price data
C = list(range(0, 15000))
df = pd.DataFrame({'season_ticket_id': A, 'average_seat_price': B, 'seat_location_id': C})
df.head()

,season_ticket_id,average_seat_price,seat_location_id
0,0,88,0
1,1,50,1
2,2,113,2
3,3,91,3
4,4,67,4


In [ ]:
# Create 45000 seat data for opening day

A = list(range(0, 45000))
B = list(random.randint(33, 177) for _ in range(45000)) # the range (33, 177) is the range price for April 1st game
df1 = pd.DataFrame({'opening_day_ticket_id': A, 'seat_price': B})
df1.head()

,opening_day_ticket_id,seat_price
0,0,156
1,1,118
2,2,123
3,3,174
4,4,62


In [ ]:
# Sort the data with the price for each table

df.sort_values(by=['average_seat_price'], ascending = False, inplace = True, ignore_index=True)
df1.sort_values(by=['seat_price'], ascending = False, inplace = True, ignore_index=True)

In [ ]:
# data for ticket holder
df

,season_ticket_id,average_seat_price
0,11897,129
1,11211,129
2,9920,129
3,13875,129
4,11683,129
...,...,...
14995,14666,20
14996,7332,20
14997,1028,20
14998,7731,20


In [ ]:
# data for available opening day ticket
df1

,opening_day_ticket_id,seat_price
0,36850,177
1,39057,177
2,13558,177
3,36291,177
4,8715,177
...,...,...
44995,17425,33
44996,1346,33
44997,28566,33
44998,17347,33


In [ ]:
# I use the function to assign opening day seats for the half season ticket holder
# First, I find the ticket with the closest price to the average ticket price for ticket holders on the opening day 
# The ticket price is also lower than the average ticket price that holders bought
# If such kind of seat exists on the opening day, I assign the seat to the ticket holder


l = 0 # start index, record the start position for available opening day ticket
season_ticket_opening = [None] * len(df) # list to record if the holder is assigned an opening day ticket
occupied = [False] * len(df1) # list to record if the opening day ticket is available

for i in range(len(df)): # for loop to assign opening day ticket for the ith season ticket holder

  value = df.loc[i].average_seat_price # the ith average price of the season ticket holder
  if value < min(df1.seat_price): # stop the loop if there is no seat for the ticket holder
    break

  for j in range(l, len(df1)): # start to search opening day available seat
    opening = df1.loc[j].seat_price # available seat price

    if opening <= value and occupied[j] == False: # if the seat is available and its value is not higher than the average season ticket price 
      season_ticket_opening[i] = df1.loc[j].opening_day_ticket_id # assign the seat to the holder
      l = j # move the start index
      occupied[j] = True # the ticket is no longer available
      break # stop to find an available seat and move to the next season ticket holder

In [ ]:
# Update the season ticket holder data

df['assign_opening_seat_id'] = season_ticket_opening
df

,season_ticket_id,average_seat_price,assign_opening_seat_id
0,11897,129,1661.0
1,11211,129,4682.0
2,9920,129,37880.0
3,13875,129,7082.0
4,11683,129,24178.0
...,...,...,...
14995,14666,20,NaN
14996,7332,20,NaN
14997,1028,20,NaN
14998,7731,20,NaN


In [ ]:
# Update opening day seat data

df1['occupied'] = occupied
df1

,opening_day_ticket_id,seat_price,occupied
0,36850,177,False
1,39057,177,False
2,13558,177,False
3,36291,177,False
4,8715,177,False
...,...,...,...
44995,17425,33,False
44996,1346,33,False
44997,28566,33,False
44998,17347,33,False


In [ ]:
# Split the season ticket dataset into two parts: one with an opening day seat and the other one without
# The idea here is that I can directly assign opening day seats to the rest of the season ticket holder

df_assign = df[~df['assign_opening_seat_id'].isna()].copy()
df_rest = df[df['assign_opening_seat_id'].isna()].copy()

In [ ]:
# Split opening day seat into three parts: not available seat, 
# available seat from the lowest price and match the amount of ticket holders without seat 
# and the rest of seats

df1_True = df1[df1['occupied'] == True].copy()
df1_other = df1[df1['occupied'] == False].iloc[:-len(df_rest), :].copy()
df1_rest = df1[df1['occupied'] == False].iloc[-len(df_rest):, :].copy()

In [ ]:
# the rest of season ticket holders without opening day seat

df_rest

,season_ticket_id,average_seat_price,assign_opening_seat_id
13275,10595,32,NaN
13276,8073,32,NaN
13277,7477,32,NaN
13278,5765,32,NaN
13279,12837,32,NaN
...,...,...,...
14995,14666,20,NaN
14996,7332,20,NaN
14997,1028,20,NaN
14998,7731,20,NaN


In [ ]:
# Same amount of the available opening day seats from the lowest price

df1_rest

,opening_day_ticket_id,seat_price,occupied
42076,4957,42,False
42077,43552,42,False
42078,20560,42,False
42079,37500,42,False
42080,11989,42,False
...,...,...,...
44995,17425,33,False
44996,1346,33,False
44997,28566,33,False
44998,17347,33,False


In [ ]:
# assign the seat

df_rest['assign_opening_seat_id'] = df1_rest['opening_day_ticket_id'].tolist()
df_rest

,season_ticket_id,average_seat_price,assign_opening_seat_id
13275,10595,32,4957
13276,8073,32,43552
13277,7477,32,20560
13278,5765,32,37500
13279,12837,32,11989
...,...,...,...
14995,14666,20,17425
14996,7332,20,1346
14997,1028,20,28566
14998,7731,20,17347


In [ ]:
# Update the available seat

df1_rest['occupied'] = [True] * len(df1_rest)
df1_rest

,opening_day_ticket_id,seat_price,occupied
42076,4957,42,True
42077,43552,42,True
42078,20560,42,True
42079,37500,42,True
42080,11989,42,True
...,...,...,...
44995,17425,33,True
44996,1346,33,True
44997,28566,33,True
44998,17347,33,True


In [ ]:
# Concatenate two parts of season ticket holders

df_final = pd.concat([df_assign, df_rest])
df_final

,season_ticket_id,average_seat_price,assign_opening_seat_id
0,11897,129,1661.0
1,11211,129,4682.0
2,9920,129,37880.0
3,13875,129,7082.0
4,11683,129,24178.0
...,...,...,...
14995,14666,20,17425.0
14996,7332,20,1346.0
14997,1028,20,28566.0
14998,7731,20,17347.0


In [ ]:
# Concatenate three parts of opening day seats

df1_final = pd.concat([df1_True, df1_rest, df1_other])
df1_final

,opening_day_ticket_id,seat_price,occupied
14823,1661,129,True
14824,4682,129,True
14825,37880,129,True
14826,7082,129,True
14827,24178,129,True
...,...,...,...
42071,4228,42,False
42072,4968,42,False
42073,35030,42,False
42074,9978,42,False


In [ ]:
# merge with two table

season_ticket_opening = df_final.merge(df1_final, left_on = 'assign_opening_seat_id', right_on = 'opening_day_ticket_id', )
season_ticket_opening

,season_ticket_id,average_seat_price,assign_opening_seat_id,opening_day_ticket_id,seat_price,occupied
0,11897,129,1661.0,1661,129,True
1,11211,129,4682.0,4682,129,True
2,9920,129,37880.0,37880,129,True
3,13875,129,7082.0,7082,129,True
4,11683,129,24178.0,24178,129,True
...,...,...,...,...,...,...
14995,14666,20,17425.0,17425,33,True
14996,7332,20,1346.0,1346,33,True
14997,1028,20,28566.0,28566,33,True
14998,7731,20,17347.0,17347,33,True


In [ ]:
# drop unnecessary columns

season_ticket_opening.drop(['assign_opening_seat_id', 'occupied'], axis = 1, inplace = True)
season_ticket_opening

,season_ticket_id,average_seat_price,opening_day_ticket_id,seat_price
0,11897,129,1661,129
1,11211,129,4682,129
2,9920,129,37880,129
3,13875,129,7082,129
4,11683,129,24178,129
...,...,...,...,...
14995,14666,20,17425,33
14996,7332,20,1346,33
14997,1028,20,28566,33
14998,7731,20,17347,33
